# Zappa
## Creating "serverless" Python web services on AWS

* Sarah Braden
* Twitter: @ifmoonwascookie
* Presented at: DesertPy Meetup 24 July 2018

How many people create APIs?

How many people here use Flask/Django/Pyramid?

How many people use AWS?

## Zappa - what is it?
Python library that helps create "Serverless" Python Web Services using AWS Lambda and API Gateway.

## What is "Serverless"? 

* There still is a server - just no "permanent infrastructure"
* With Zappa, each request is given its own virtual HTTP "server" by Amazon API Gateway
* AWS handles the horizontal scaling automatically 
* only pay for the milliseconds of server time you use

## How does it work?
1. Each request calls your application from a memory cache in AWS Lambda
2. Return the response via Python's Web Server Gateway Interface (WSGI)
3. After your app returns, the "server" dies.

## Getting Started - AWS Stuff

Do not pass go unless:
1. you have a valid AWS account
2. your AWS credentials file is properly installed

## Getting Started - Installation
```
$ mkvirtualenv zappa-env
$ pip install zappa
```

## Getting Started - Configuration

Make a configuration file called ```zappa_settings.json``` in your project directory using the command:

```$ zappa init```

This command automatically detects your application type (Flask/Django) defines your basic deployment settings.

## Getting Started - Configuration
```
{
    "dev": {
        "app_function": "my_app.app",
        "aws_region": "us-west-1",
        "profile_name": "sarah-aws",
        "project_name": "my-zappa-project",    
        "s3_bucket": "zappa-fsdf980",
        "runtime": "python3.6",
        "manage_roles": true        
    }
}
```

you can have as many stages as you like: dev, staging, and production.

## What does a Flask app look like?

In [ ]:
# my_app.py

from my_stuff import run_model, get_wav_from_s3
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/wavs/<string:wav_filename>')
def do_things(wav_filename=None):
    
    get_wav_from_s3(wav_filename)
    
    response = jsonify(
        wavfile = wav_filename,
        prediction = run_model('/tmp/' + wav_filename),
    )
    response.status_code = 200
    
    return response 

if __name__ == '__main__':
    app.run()

## Deploy time!

```
$ zappa deploy dev
Deploying..
Your application is now live at: https://7k6anj0k99.execute-api.us-east-1.amazonaws.com/dev
```

## What just happened?
* automatically package up your application and local virtual environment into a Lambda-compatible archive
* replace any dependencies with versions precompiled for Lambda
* set up the function handler and necessary WSGI Middleware
* upload the archive to S3
* create and manage the necessary Amazon IAM policies and roles

## What just happened?
* register it as a new Lambda function
* create a new API Gateway resource
* create WSGI-compatible routes for it
* link it to the new Lambda function
* finally delete the archive from your S3 bucket

## Other useful Zappa commands

```$ zappa update dev```

Upload new Python code and doesn't change anything else.

```$ zappa undeploy dev```

Remove the API Gateway and Lambda function.

```$ zappa package dev```

Build the application package without actually uploading and registering it as a Lambda function.


## IAM role gotchas

In [ ]:
{
    "dev": {
        "app_function": "my_app.app",
        "aws_region": "us-west-1",
        "profile_name": "sarah-aws",
        "project_name": "my-zappa-project",    
        "s3_bucket": "zappa-fsdf980",
        "runtime": "python3.6",
        "manage_roles": true        
    }
}


In [ ]:
{
    "dev": {
        "app_function": "my_app.app",
        "aws_region": "us-west-1",
        "profile_name": "sarah-aws",
        "project_name": "my-zappa-project",    
        "s3_bucket": "zappa-fsdf980",
        "runtime": "python3.6",
        "manage_roles": false,
        "role_name": "dev_lambda_role"
    }
}

## Trust Policy

In [ ]:
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": [
            "lambda.amazonaws.com",
            "apigateway.amazonaws.com",
            "events.amazonaws.com"
        ]
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

// Name of Zappa execution role. Default <project_name>-<env>-ZappaExecutionRole. To use a different, pre-existing policy, you must also set manage_roles to false.

"slim_handler": true,  // Useful if project >50M. Set true to just upload a small handler to Lambda and load actual project from S3 at runtime. Default false.